# Metrics (Text)

In [456]:
import re
import pandas as pd

###  *****   Config *****
config = {
    "json_file_path": "/Users/zixuanzhao/Desktop/Metrics/GroundTruth/GroundTruth_Strengthening-Mechanism.json",    ### input_json
    "csv_file_path": "/Users/zixuanzhao/Desktop/Metrics/Structured-Info/Text/Spark-4.0Ultra.csv",  ### input_csv
    "output_csv_path": "/Users/zixuanzhao/Desktop/Metrics/GroundTruth/GroundTruth_Strengthening-Mechanism.csv",  ### output_csv
    "output_columns": {
        # "groundtruth_column": "GroundTruth_Composition",  ### output_csv_groundtruth_column
        # "extracted_column": "Extracted_Composition",   ### output_csv_extracted_column
        # "id_column": "ID", ### output_csv_id_column
        # "csv_column": "Composition" ### input_csv_extracted_column
        
        # "groundtruth_column": "GroundTruth_Heat_treatment",  ### output_csv_groundtruth_column
        # "extracted_column": "Extracted_Heat_treatment",   ### output_csv_extracted_column
        # "id_column": "ID", ### output_csv_id_column
        # "csv_column": "Heat_Treatment" ### input_csv_extracted_column
        
        # "groundtruth_column": "GroundTruth_Heat_treatment2",  ### output_csv_groundtruth_column
        # "extracted_column": "Extracted_Heat_treatment2",   ### output_csv_extracted_column
        # "id_column": "ID", ### output_csv_id_column
        # "csv_column": "Deformation/Manufacturing_Technique" ### input_csv_extracted_column
        
        # "groundtruth_column": "GroundTruth_Microstructure1",  ### output_csv_groundtruth_column
        # "extracted_column": "Extracted_Microstructure1",   ### output_csv_extracted_column
        # "id_column": "ID", ### output_csv_id_column
        # "csv_column": "Microstructure1" ### input_csv_extracted_column
        
        # "groundtruth_column": "GroundTruth_Microstructure2",  ### output_csv_groundtruth_column
        # "extracted_column": "Extracted_Microstructure2",   ### output_csv_extracted_column
        # "id_column": "ID", ### output_csv_id_column
        # "csv_column": "Microstructure2" ### input_csv_extracted_column
        
        # "groundtruth_column": "GroundTruth_Properties",  ### output_csv_groundtruth_column
        # "extracted_column": "Extracted_Properties",   ### output_csv_extracted_column
        # "id_column": "ID", ### output_csv_id_column
        # "csv_column": "Properties" ### input_csv_extracted_column
        
        # "groundtruth_column": "GroundTruth_Performance",  ### output_csv_groundtruth_column
        # "extracted_column": "Extracted_Performance",   ### output_csv_extracted_column
        # "id_column": "ID", ### output_csv_id_column
        # "csv_column": "Performance" ### input_csv_extracted_column
        
        "groundtruth_column": "GroundTruth_Strengthening-Mechanism",  ### output_csv_groundtruth_column
        "extracted_column": "Extracted_Strengthening-Mechanism",   ### output_csv_extracted_column
        "id_column": "ID", ### output_csv_id_column
        "csv_column": "Strengthening_Mechanism" ### input_csv_extracted_column
    },
    "line_starts_with": {
        "id": '{{"ID":',  ### input_json_ID_indictor
        # "groundtruth": '{"Structured_Composition":', ### input_json_Structured_Composition_indictor
        # "groundtruth": '{"Heat_Treatment":', ### input_json_Structured_Processing_indictor
        # "groundtruth": '{"Deformation/Manufacturing_Technique":', ### input_json_Structured_Deformation_indictor
        # "groundtruth": '{"Structured_Microstructure1":', ### input_json_Structured_Microstructure1_indictor
        # "groundtruth": '{"Structured_Microstructure2":', ### input_json_Structured_Microstructure2_indictor
        # "groundtruth": '{"Structured_Properties":', ### input_json_Structured_Properties_indictor
        # "groundtruth": '{"Structured_Performance":', ### input_json_Structured_Performance_indictor
        "groundtruth": '{"Structured_Strengthening_Mechanism":', ### input_json_Structured_Strengthening-Mechanism_indictor
    },
    "id_pattern": r'"ID":\s*(\d+)',  # Pattern to extract ID
    "json_block_pattern": r'```(.*?)```', # New pattern to extract JSON blocks between ```{ and }```
    # "json_extraction_pattern": r'"Structured_Composition":\s*"(.*?)"}}', # Pattern to extract Structured_Composition
    # "json_extraction_pattern": r'\{"Heat_Treatment":(.*?)(?:,"Deformation/Manufacturing_Technique"|$)|^N/A$', # Pattern to extract Heat_Treatment
    # "json_extraction_pattern": r'\{"Deformation/Manufacturing_Technique":(.*?)(?:}}|$)|^N/A$', # Pattern to extract Structured_Deformation
    # "json_extraction_pattern": r'\{"Structured_Microstructure1":(.*?)(?:}}|$)|^N/A$', # Pattern to extract Structured_Microstructure1
    # "json_extraction_pattern": r'\{"Structured_Microstructure2":(.*?)(?:}}|$)|^N/A$', # Pattern to extract Structured_Microstructure2
    # "json_extraction_pattern": r'\{"Structured_Properties":(.*?)(?:,"Structured_ Performance"|$)|^N/A$', # Pattern to extract Properties
    # "json_extraction_pattern": r'\{"Structured_ Performance":(.*?)(?:}}|$)|^N/A$', # Pattern to extract Structured_Performance
    "json_extraction_pattern": r'\{"Structured_Strengthening_Mechanism":(.*?)(?:}}|$)|^N/A$', # Pattern to extract Structured_Strengthening-Mechanism
        
    "preprocess_patterns": [
        (r'"[^"]+":"{}"', ''),
        (r'"[^"]+":"N/A"', ''),
        (r'"[^"]+":"{N/A}"', ''),
        (r'{value/unit}', 'N/A'),
        (r'"( Density|Melting point|Electrical Conductivity|Electrical resistivity|Magnetic Properties|Saturation Magnetization/Ms|Thermodynamic Properties|Mixing Enthalpy|Curie Temperature|Object|Transformation temperature|Mixing Enthalpy|Mixing Entropy|Heat Capacity|Thermal conductivity|Thermal expansion|Thermal diffusivity|Remanent Magnetic Induction/Br|Coercivity/Hc|Saturation Magnetization/M|Magnetization Finish/Mf|Description|YS|UTS|Elongation|Compressive Strength|Compressive strain|Fracture strength|Fracture strain|Reduction of area|Hardness|Young’s modulus|Shear modulus|Flexural modulus|Poisson’s ratio)": \{\},?\s*', ''), 
        (r'"Mechanical Properties": {N/A}', ''),
        (r'"Physical Properties": {N/A}', ''),
        (r'"Magnetic Properties": {N/A}', ''),
        (r'"Thermodynamic Properties": {N/A}', ''),
        (r'"Mechanical Properties":{N/A}', ''),
        (r'"Physical Properties":{N/A}', ''),
        (r'"Magnetic Properties":{N/A}', ''),
        (r'"Thermodynamic Properties":{N/A}', ''),
        (r'"Mechanical Properties":"{N/A}"', ''),
        (r'"Physical Properties":"{N/A}"', ''),
        (r'"Magnetic Properties":"{N/A}"', ''),
        (r'"Thermodynamic Properties":"{N/A}"', ''),
        (r'"Properties": "N/A"', 'N/A'),
        (r'"Mechanical Properties":"{}"', ''),
        (r'"Physical Properties":"{}"', 'N/A'),
        (r'"Magnetic Properties":"{}"', 'N/A'),
        (r'"Thermodynamic Properties":"{}"', 'N/A'),        
        (r'"Others":"\{.*?\}",?', ''),
        (r'"Others":\s*\{.*?\},?',''),
        (r'"Others":"\{.*?\}",?', ''),                                  
        (r'"Others":"\{.*?\}",?', ''),
        (r'"Others":"N/A"', ''),
        (r'"Others": "N/A"', ''),
        (r'"Others":"[^"]*",?', ''),
        (r'"Other Properties":"\{.*?\}",?', ''),
        (r'"Other Properties":\s*\{.*?\},?',''),
        (r'"Other Properties":"\{.*?\}",?', ''),                                  
        (r'"Other Properties":"\{.*?\}",?', ''),
        (r'"Other Properties":"N/A"', ''),
        (r'"Other Properties": "N/A"', ''),
        (r'"Other Properties":"[^"]*",?', ''),
        (r'"Property Name":"\{.*?\}",?', ''),
        (r'"Property Name":\s*\{.*?\},?',''),
        (r'"Property Name":"\{.*?\}",?', ''),                                  
        (r'"Property Name":"\{.*?\}",?', ''),
        (r'"Property Name":"N/A"', ''),
        (r'"Property Name": "N/A"', ''),
        (r'"Property Name":"[^"]*",?', ''),
        (r'"Property Value":"\{.*?\}",?', ''),
        (r'"Property Value":\s*\{.*?\},?',''),
        (r'"Property Value":"\{.*?\}",?', ''),                                  
        (r'"Property Value":"\{.*?\}",?', ''),
        (r'"Property Value":"N/A"', ''),
        (r'"Property Value": "N/A"', ''),
        (r'"Property Value":"[^"]*",?', ''),
        (r'"Other Parameters":"\{.*?\}",?', ''),
        (r'"Description":"{N/A}"', ''),  
        (r'"Characteristic":\s*".*?",?', ''),
        (r'"Solute-Atoms": "N/A",', ''),
        (r'"Grain Characteristics": "N/A",', ''),
        (r'"Dislocation": "N/A",', ''),
        (r'"Defect": "N/A",', ''),
        (r'"Texture": "N/A",', ''),
        (r'"Microstructure": "N/A",', ''),
        (r'"Zones and Deformation Bands": "N/A",', ''),
        (r'"Fracture": "N/A",', ''),
        (r'"Crack": "N/A"', ''),
        (r'"Grain Boundary": "N/A",', ''),
        (r'"Segregation": "N/A",', ''),
        (r'{}', 'N/A'),
        (r'Here\s+(is|are)\s+.*\s+(JSON\s+format|output|text|treatment|representation)\s*:', ''),
        (r'Here\s+(is|are)\s+.*\s+(JSON)\s*:', ''),
        (r'(Based on)\s+the\s+.*\s+(provided\s+text|information)\s*:', ''),
        (r'Based\s+on\s+.*\s+(JSON\s+format|output|text)\s*:', ''),
        (r'Here\s+(is|are)\s+the\s+customized\s+JSON\s+for\s+.*?:', ''),   
        (r'Note:\s+.*?\..*?\.', ''),
        (r'Note:\s+.*?\.', ''),
        (r'"Melting": {"Method": "N/A","Equipment": "N/A","Temperature": "N/A","Atmosphere": "N/A"}', ''),    
        (r'"Melting": {"Method": "N/A","Equipment": "N/A","Temperature": "N/A"}', ''),
        (r'"Casting": {"Method": "N/A","Temperature": "N/A","Diameter": "N/A"}', ''),        
        (r'"Austenitization": {"Temperature": "N/A","Time": "N/A"}', ''),
        (r'"Transformation": {"Temperature": "N/A","Time": "N/A"}', ''),        
        (r'"Homogenization": {"Temperature": "N/A","Time": "N/A"},', ''),
        (r'"Solution": {"Temperature": "N/A","Time": "N/A","HeatingRate": "N/A"}', ''),        
        (r'"Isothermal Aging": {"Temperature": "N/A","Time": "N/A"}', ''),
        (r'"Pre-Aging": {"Temperature": "N/A","Time": "N/A"}', ''),        
        (r'"Retrogression": {"Temperature": "N/A","Time": "N/A"}', ''),
        (r'"Re-aging": {"Temperature": "N/A","Time": "N/A"}', ''),            
        (r'"Natural Aging": {"Temperature": "N/A","Time": "N/A"}', ''),
        (r'"Creep Aging": {"Temperature": "N/A","Time": "N/A","Stress": "N/A"}', ''), 
        (r'"Melting": {"Method": "{N/A}","Equipment": "{N/A}","Temperature": "{N/A}","Atmosphere": "{N/A}"}', ''),
        (r'"Casting": {"Method": "{N/A}","Temperature": "{N/A}","Diameter": "{N/A}"}', ''), 
        (r'"Austenitization": {"Temperature": "{N/A}","Time": "{N/A}"}', ''),
        (r'"Transformation": {"Temperature": "{N/A}","Time": "{N/A}"}', ''), 
        (r'"Homogenization": {"Temperature": "{N/A}","Time": "{N/A}"},', ''),
        (r'"Solution": {"Temperature": "{N/A}","Time": "{N/A}","HeatingRate": "{N/A}"}', ''), 
        (r'"Isothermal Aging": {"Temperature": "{N/A}","Time": "{N/A}"}', ''),
        (r'"Pre-Aging": {"Temperature": "{N/A}","Time": "{N/A}"}', ''), 
        (r'"Retrogression": {"Temperature": "{N/A}","Time": "{N/A}"}', ''),
        (r'"Re-aging": {"Temperature": "{N/A}","Time": "{N/A}"}', ''), 
        (r'"Natural Aging": {"Temperature": "{N/A}","Time": "{N/A}"}', ''),
        (r'"Creep Aging": {"Temperature": "{{N/A}}","Time": "{{N/A}}","Stress": "{N/A}"}', ''),  
        (r'"Melting": {"Method": "N/A", "Equipment": "N/A", "Temperature": "N/A", "Atmosphere": "N/A"}', ''),    
        (r'"Melting": {"Method": "N/A", "Equipment": "N/A", "Temperature": "N/A"}', ''),
        (r'"Casting": {"Method": "N/A", "Temperature": "N/A", "Diameter": "N/A"}', ''),        
        (r'"Austenitization": {"Temperature": "N/A", "Time": "N/A"}', ''),
        (r'"Transformation": {"Temperature": "N/A", "Time": "N/A"}', ''),        
        (r'"Homogenization": {"Temperature": "N/A", "Time": "N/A"},', ''),
        (r'"Solution": {"Temperature": "N/A", "Time": "N/A", "HeatingRate": "N/A"}', ''),        
        (r'"Isothermal Aging": {"Temperature": "N/A", "Time": "N/A"}', ''),
        (r'"Pre-Aging": {"Temperature": "N/A", "Time": "N/A"}', ''),        
        (r'"Retrogression": {"Temperature": "N/A", "Time": "N/A"}', ''),
        (r'"Re-aging": {"Temperature": "N/A", "Time": "N/A"}', ''),            
        (r'"Natural Aging": {"Temperature": "N/A", "Time": "N/A"}', ''),
        (r'"Creep Aging": {"Temperature": "N/A", "Time": "N/A", "Stress": "N/A"}', ''), 
        (r'"Melting": {"Method": "{N/A}","Equipment": "{N/A}","Temperature": "{N/A}", "Atmosphere": "{N/A}"}', ''),
        (r'"Casting": {"Method": "{N/A}","Temperature": "{N/A}", "Diameter": "{N/A}"}', ''), 
        (r'"Austenitization": {"Temperature": "{N/A}", "Time": "{N/A}"}', ''),
        (r'"Transformation": {"Temperature": "{N/A}", "Time": "{N/A}"}', ''), 
        (r'"Homogenization": {"Temperature": "{N/A}", "Time": "{N/A}"},', ''),
        (r'"Solution": {"Temperature": "{N/A}", "Time": "{N/A}", "HeatingRate": "{N/A}"}', ''), 
        (r'"Isothermal Aging": {"Temperature": "{N/A}", "Time": "{N/A}"}', ''),
        (r'"Pre-Aging": {"Temperature": "{N/A}", "Time": "{N/A}"}', ''), 
        (r'"Retrogression": {"Temperature": "{N/A}", "Time": "{N/A}"}', ''),
        (r'"Re-aging": {"Temperature": "{N/A}", "Time": "{N/A}"}', ''), 
        (r'"Natural Aging": {"Temperature": "{N/A}", "Time": "{N/A}"}', ''),
        (r'"Creep Aging": {"Temperature": "{{N/A}}", "Time": "{{N/A}}", "Stress": "{N/A}"}', ''),     
        (r'"Melting": N/A', ''),
        (r'"Casting": N/A', ''),        
        (r'"Austenitization": N/A', ''),
        (r'"Transformation": N/A', ''),        
        (r'"Homogenization": N/A', ''), 
        (r'"Solution": N/A', ''),         
        (r'"Isothermal Aging": N/A', ''), 
        (r'"Pre-Aging": N/A', ''),         
        (r'"Retrogression": N/A', ''), 
        (r'"Re-aging": N/A', ''),             
        (r'"Natural Aging": N/A', ''), 
        (r'"Creep Aging": N/A', ''),  
        (r'"Melting": N/A', ''), 
        (r'"Casting": N/A', ''),  
        (r'"Austenitization": N/A', ''), 
        (r'"Transformation": N/A', ''),  
        (r'"Homogenization": N/A', ''), 
        (r'"Solution": N/A', ''),  
        (r'"Isothermal Aging": N/A', ''), 
        (r'"Pre-Aging": N/A', ''),  
        (r'"Retrogression": N/A', ''), 
        (r'"Re-aging": N/A', ''),  
        (r'"Natural Aging": N/A', ''), 
        (r'"Creep Aging": N/A', ''),    
        (r'"Temperature": "{N/A}"', ''),
        (r'"Time": "{N/A}"', ''), 
        (r'"Rotation speed": "N/A"', ''), 
        (r'"Processing speed": "N/A"', ''), 
        (r'"Laser Processing": "N/A"', ''), 
        (r'"Laser Power": "N/A"', ''), 
        (r'"Rotation speed": "N/A"', ''),     
        (r'"Natural Aging": {"Temperature": "{N/A}", "Time": "{N/A}"}', ''),
        (r'"Stress": "{N/A}"', ''),                       
        (r'"Alloy Formula":', ''),
        (r'"States":', ''),
        (r'"Composition":', ''),
        (r'"Sample Name":', ''),
        (r'"Alloy":', ''),
        (r'"Method":', ''),
        (r'"Name":', ''),
        (r'"Temperature":', ''),  
        (r'"Cooling": {"N/A"}', ''),
        (r'"Quenching": {"N/A"}', ''),                                                      
        (r'"Time":', ''),
        (r'"HeatingRate":', ''),
        (r'"Diameter":', ''),
        (r'"Equipment":', ''),
        (r'"Atmosphere":', ''),
        (r'"Diameter":', ''),
        (r'"Stress":', ''),
        (r'"Heat treatment":', ''),
        (r'"Other Deformation":', ''),
        (r'"Other Manufacturing/Preparation Processing":', ''),
        (r'"Heat_Treatment":', ''),
        (r'"Strengthening Mechanism":', ''),
        (r'"Strengthening Phase":', ''),
        (r'"Strengthening mechanism and proportion":', ''),
        (r'"Strengthening Effect":', ''),
        (r'"Heat_Treatment":', ''),
        (r'"Heat_Treatment":', ''),
        (r'"Heat_Treatment":', ''),
        (r'"Heat_Treatment":', ''),
        (r'"Heat_Treatment":', ''),
        (r'"Heat_Treatment":', ''),        
        (r'"Other Parameters":', ''),
        (r'"{Casting}"', ''),  # Deformation/Manufacturing_Technique
        (r'"{Solution-treated}"', ''), # Deformation/Manufacturing_Technique
        (r'"{Water Quenching}"', ''),  # Deformation/Manufacturing_Technique
        (r'"{Aging/Ageing}"', ''),  # Deformation/Manufacturing_Technique
        (r'"{Homogenizing}"', ''),        
        (r'\{.*? cooling\}', ''),
        (r'\{.*? Cooling\}', ''),
        (r'\{.*? quenching\}', ''),
        (r'\{.*? Quenching\}', ''),
        (r'"Microstructure":', ''),
        (r'"Description":"\{.*?\}",?', ''),
        (r'"Description":"N/A",?', ''),
        (r'"Description": "N/A"', ''),
        (r'"Description":"[^"]*",?', ''),
        (r'"Precipitate":', ''),   
        (r'"Solute-Atoms":', ''),
        (r'"Grain Boundary":', ''),   
        (r'"Grain Characteristics":', ''),
        (r'"Dislocation":', ''),   
        (r'"Defect":', ''),
        (r'"Texture":', ''),   
        (r'"Segregation":', ''), 
        (r'"Zones and Deformation Bands":', ''),   
        (r'"Fracture":', ''),
        (r'"Crack":', ''),
        (r'"Description": "\{.*?\}",?', ''),
        (r'"Description": "N/A",?', ''),
        (r'"Description": "[^"]*",?', ''),
        (r'"Characteristic":"\{.*?\}",?', ''),
        (r'"Characteristic":"N/A",?', ''),
        (r'"Characteristic":"[^"]*",?', ''),
        (r'"States":"{N/A}"', ''),
        (r'"Mixing Enthalpy":"{N/A}"', ''),
        (r'"Mixing Entropy":"{N/A}"', ''),
        (r'"Heat Capacity":"{N/A}"', ''),
        (r'"Thermal conductivity":"{N/A}"', ''),
        (r'"Thermal expansion":"{N/A}"', ''),
        (r'"Thermal diffusivity":"{N/A}"', ''),
        (r'"Curie Temperature": "{N/A}"', ''),
        (r'"Transformation temperature":"{N/A}"', ''),
        (r'"Characteristic":"[^"]*",?', ''),
        (r'"Characteristic":"[^"]*",?', ''),
        (r'json', ''),
        (r'"\d+"', ''),
        (r':', ' '),
        (r'\\', ' '),
        (r'"Other Properties"', ' '),
        (r',', ' ')          
    ],
}
###  *****   Config *****

# Function to extract json block
def extract_json_block(text):
    """
    Extracts JSON content between ```{ and }``` markers.
    Returns the content without the markers.
    """
    if not isinstance(text, str):
        return text
    
    pattern = config["json_block_pattern"]
    match = re.search(pattern, text, re.DOTALL)  # re.DOTALL allows . to match newlines
    if match:
        return match.group(1).strip()
    return text

# Function to calculate metrics for a single row
def calculate_metrics(row):
    def tokenize(text):
        if not isinstance(text, str) or not text.strip():
            return []
        tokens = []
        for item in re.split(r'[:,]', text.strip()):
            tokens.extend([t.strip() for t in item.split() if t.strip()])
        return tokens

    groundtruth_tokens = tokenize(row[config["output_columns"]["groundtruth_column"]])
    extracted_tokens = tokenize(row[config["output_columns"]["extracted_column"]])

    # If both are empty, return perfect metrics
    if not groundtruth_tokens and not extracted_tokens:
        return 1, 0, 0  # TP, FP, FN

    true_positives = 0
    used_positions = set()

    for i, gt_token in enumerate(groundtruth_tokens):
        for j, ex_token in enumerate(extracted_tokens):
            if j not in used_positions and gt_token == ex_token:
                true_positives += 1
                used_positions.add(j)
                break

    false_positives = len(extracted_tokens) - true_positives
    false_negatives = len(groundtruth_tokens) - true_positives

    return true_positives, false_positives, false_negatives

def calculate_char_metrics(row):
    # Get strings
    groundtruth = str(row[config["output_columns"]["groundtruth_column"]]).strip()
    extracted = str(row[config["output_columns"]["extracted_column"]]).strip()
    
    # If both are empty, return perfect metrics
    if not groundtruth and not extracted:
        return 1, 0, 0  # TP, FP, FN
    
    # Create character count dictionaries
    gt_chars = {}
    ex_chars = {}
    
    # Count occurrences of each character (ignoring whitespace and punctuation)
    for char in groundtruth:
        if char.strip() and not char in ',./:;(){}[]':
            gt_chars[char] = gt_chars.get(char, 0) + 1
            
    for char in extracted:
        if char.strip() and not char in ',./:;(){}[]':
            ex_chars[char] = ex_chars.get(char, 0) + 1
    
    # Calculate true positives
    true_positives = 0
    for char, count in gt_chars.items():
        if char in ex_chars:
            true_positives += min(count, ex_chars[char])
    
    # Calculate false positives and false negatives
    false_positives = sum(ex_chars.values()) - true_positives
    false_negatives = sum(gt_chars.values()) - true_positives
    
    return true_positives, false_positives, false_negatives

# Function to calculate metrics for all rows
def calculate_row_metrics(df):
    results = []
    total_tp, total_fp, total_fn = 0, 0, 0
    total_char_tp, total_char_fp, total_char_fn = 0, 0, 0
    
    for _, row in df.iterrows():
        # Token-level metrics
        tp, fp, fn = calculate_metrics(row)
        # Character-level metrics
        char_tp, char_fp, char_fn = calculate_char_metrics(row)
        
        total_tp += tp
        total_fp += fp
        total_fn += fn
        total_char_tp += char_tp
        total_char_fp += char_fp
        total_char_fn += char_fn

        # Calculate token-level metrics
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        accuracy = tp / (tp + fp + fn) if (tp + fp + fn) > 0 else 0

        # Calculate character-level metrics
        char_precision = char_tp / (char_tp + char_fp) if (char_tp + char_fp) > 0 else 0
        char_recall = char_tp / (char_tp + char_fn) if (char_tp + char_fn) > 0 else 0
        char_f1 = (2 * char_precision * char_recall) / (char_precision + char_recall) if (char_precision + char_recall) > 0 else 0
        char_accuracy = char_tp / (char_tp + char_fp + char_fn) if (char_tp + char_fp + char_fn) > 0 else 0

        results.append({
            config["output_columns"]["id_column"]: row[config["output_columns"]["id_column"]],
            "GroundTruth": row[config["output_columns"]["groundtruth_column"]],
            "Extracted": row[config["output_columns"]["extracted_column"]],
            "Token_Precision": precision,
            "Token_Recall": recall,
            "Token_F1": f1,
            "Token_Accuracy": accuracy,
            "Token_TP": tp,
            "Token_FP": fp,
            "Token_FN": fn,
            "Char_Precision": char_precision,
            "Char_Recall": char_recall,
            "Char_F1": char_f1,
            "Char_Accuracy": char_accuracy,
            "Char_TP": char_tp,
            "Char_FP": char_fp,
            "Char_FN": char_fn
        })

    # Calculate overall token-level metrics
    total_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    total_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
    total_f1 = (2 * total_precision * total_recall) / (total_precision + total_recall) if (total_precision + total_recall) > 0 else 0
    total_accuracy = total_tp / (total_tp + total_fp + total_fn) if (total_tp + total_fp + total_fn) > 0 else 0

    # Calculate overall character-level metrics
    total_char_precision = total_char_tp / (total_char_tp + total_char_fp) if (total_char_tp + total_char_fp) > 0 else 0
    total_char_recall = total_char_tp / (total_char_tp + total_char_fn) if (total_char_tp + total_char_fn) > 0 else 0
    total_char_f1 = (2 * total_char_precision * total_char_recall) / (total_char_precision + total_char_recall) if (total_char_precision + total_char_recall) > 0 else 0
    total_char_accuracy = total_char_tp / (total_char_tp + total_char_fp + total_char_fn) if (total_char_tp + total_char_fp + total_char_fn) > 0 else 0

    total_metrics = {
        "Token_Precision": total_precision,
        "Token_Recall": total_recall,
        "Token_F1": total_f1,
        "Token_Accuracy": total_accuracy,
        "Token_TP": total_tp,
        "Token_FP": total_fp,
        "Token_FN": total_fn,
        "Char_Precision": total_char_precision,
        "Char_Recall": total_char_recall,
        "Char_F1": total_char_f1,
        "Char_Accuracy": total_char_accuracy,
        "Char_TP": total_char_tp,
        "Char_FP": total_char_fp,
        "Char_FN": total_char_fn
    }
    
    return results, total_metrics

# Main Function
def main():
    # Extract GroundTruth data
    rows = []
    with open(config["json_file_path"], 'r', encoding='utf-8') as infile:
        current_row = {config["output_columns"]["id_column"]: "", config["output_columns"]["groundtruth_column"]: ""}
        for line in infile:
            line = line.strip()
            if not line:
                continue
                
            # Check for ID line
            if line.startswith(config["line_starts_with"]["id"]):
                id_match = re.search(config["id_pattern"], line)
                if id_match:
                    current_row[config["output_columns"]["id_column"]] = id_match.group(1)

            # Check for GroundTruth composition line
            elif line.startswith(config["line_starts_with"]["groundtruth"]):
                composition_match = re.search(config["json_extraction_pattern"], line)
                if composition_match:
                    current_row[config["output_columns"]["groundtruth_column"]] = composition_match.group(1)
                rows.append(current_row)
                current_row = {config["output_columns"]["id_column"]: "", config["output_columns"]["groundtruth_column"]: ""}

    groundtruth_df = pd.DataFrame(rows)

    # Load extracted data from CSV
    extracted_df = pd.read_csv(config["csv_file_path"])
    if config["output_columns"]["csv_column"] in extracted_df.columns:
        groundtruth_df[config["output_columns"]["extracted_column"]] = extracted_df[config["output_columns"]["csv_column"]]
    else:
        raise ValueError("The input CSV file must contain the specified column!")

    # Preprocess content
    def preprocess(content):
        if not isinstance(content, str):
            return content
        
        # First extract any JSON blocks
        content = extract_json_block(content)
        
        for pattern, replacement in config["preprocess_patterns"]:
            content = re.sub(pattern, replacement, content)
        return content.strip().replace('"', '').replace('{', '').replace('}', '')

    groundtruth_df[config["output_columns"]["groundtruth_column"]] = groundtruth_df[config["output_columns"]["groundtruth_column"]].apply(preprocess)
    groundtruth_df[config["output_columns"]["extracted_column"]] = groundtruth_df[config["output_columns"]["extracted_column"]].apply(preprocess)
    
    # Save comparison to CSV
    comparison_df = groundtruth_df[
        [
            config["output_columns"]["id_column"],
            config["output_columns"]["groundtruth_column"],
            config["output_columns"]["extracted_column"]
        ]
    ]
    comparison_df.to_csv(config["output_csv_path"], index=False)

    # Calculate metrics
    results, total_metrics = calculate_row_metrics(groundtruth_df)
    
    # Print results for each row
    # print("Results for each row:")
    # for result in results:
    #     print(f"ID: {result[config['output_columns']['id_column']]}")
    #     print(f"GroundTruth: {result['GroundTruth']}")
    #     print(f"Extracted: {result['Extracted']}")
    #     print(f"Token Precision: {result['Token_Precision']:.4f}")
    #     print(f"Token Recall: {result['Token_Recall']:.4f}")
    #     print(f"Token F1: {result['Token_F1']:.4f}")
    #     print(f"Token Accuracy: {result['Token_Accuracy']:.4f}")
    #     print(f"Token TP: {result['Token_TP']}, FP: {result['Token_FP']}, FN: {result['Token_FN']}")
    #     print("-" * 50)
    
    print("\nOverall Results:")
    print("Token-level Metrics:")
    print(f"Token Precision: {total_metrics['Token_Precision']:.4f}")
    print(f"Token Recall: {total_metrics['Token_Recall']:.4f}")
    print(f"Token F1-Score: {total_metrics['Token_F1']:.4f}")
    print(f"Token Accuracy: {total_metrics['Token_Accuracy']:.4f}")
    print(f"Token TP: {total_metrics['Token_TP']}, FP: {total_metrics['Token_FP']}, FN: {total_metrics['Token_FN']}")

    print("\nCharacter-level Metrics:")
    print(f"Char Precision: {total_metrics['Char_Precision']:.4f}")
    print(f"Char Recall: {total_metrics['Char_Recall']:.4f}")
    print(f"Char F1-Score: {total_metrics['Char_F1']:.4f}")
    print(f"Char Accuracy: {total_metrics['Char_Accuracy']:.4f}")
    print(f"Char TP: {total_metrics['Char_TP']}, FP: {total_metrics['Char_FP']}, FN: {total_metrics['Char_FN']}")

if __name__ == "__main__":
    main ()


Overall Results:
Token-level Metrics:
Token Precision: 0.8018
Token Recall: 0.8593
Token F1-Score: 0.8295
Token Accuracy: 0.7087
Token TP: 635, FP: 157, FN: 104

Character-level Metrics:
Char Precision: 0.8957
Char Recall: 0.9276
Char F1-Score: 0.9114
Char Accuracy: 0.8372
Char TP: 3832, FP: 446, FN: 299


/var/folders/jt/2gvzynk95l5bq44vwtsyyqrm0000gn/T/ipykernel_29377/1412820351.py:503: DtypeWarning: Columns (1,2,3,4,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  extracted_df = pd.read_csv(config["csv_file_path"])


# Metrics (Table)

In [ ]:
import os
import pandas as pd
import csv
import json
import re

### ***** Config *****
config = {
    # 文件路径配置
    "json_dir1": "/Users/zixuanzhao/Desktop/Metrics/Structured-Info/GroundTruth/Table/GroundTruth",
    "json_dir2": "/Users/zixuanzhao/Desktop/Metrics/Structured-Info/Results/Table/SparkMax",
    "csv_output_path": "/Users/zixuanzhao/Desktop/Metrics/Structured-Info/Results/Table/output.csv",
    # 列名配置
    "output_columns": {
        "filename1_column": "GroundTruth-DOI",
        "content1_column": "GroundTruth",
        "filename2_column": "Extraction-DOI",
        "content2_column": "Extraction"
    },
    # 需要替换的内容模式
    "preprocess_patterns": [
        (r'Here is the JSON output.*?(format|input)', ''),  # 去掉 "Here is the JSON output" 开头到 "format" 为止的部分
        (r'Here is the output.*?(format|input)', ''),
        (r'Here is.*?(format|input)', ''),
        (r'/', ''),  # 去掉所有单斜杠 `/`
        (r'\\', ''),  # 去掉所有反斜杠 `\`
        (r'{', ''),  # 去掉左花括号 `{`
        (r'}', ''),  # 去掉右花括号 `}`
        (r'```json', ''),  # 替换 ```json 为空
        (r'.json', ''),  # 替换 .json 为空
        (r'```', ''),  # 替换 ``` 为空
        (r'\\n', ''),  # 替换 `\n` 为空
        (r'\n', ''),  # 替换换行符为空
        (r'/n', ''),  # 替换换行符为空        
        (r'nnn', ''),  # 替换换行符为空
        (r'nn', ''),  # 替换换行符为空
        (r'n###', '###'),  # 替换换行符为空
        (r':', ''),  # 替换冒号为空
        (r',', ''),  # 替换冒号为空
        (r"'", ''), # 替换单引号为空                                    
        (r'"', '')  # 替换双引号为空
    ],
    # 需要替换的内容模式（仅用于第二列）
    "preprocess_patterns2": [
        (r'###<[^>]+>', '###<')  # 替换所有 ###<...> 中第一个尖括号里的内容
    ],
    # 需要替换的内容模式（用于第二列和第四列）
    "preprocess_patterns3": [
        (r'###', ' '),  # 替换 ### 为空格
        (r'<', ' '),  # 替换 < 为空格
        (r'=', ' '),  # 替换 = 为空格
        (r'>', ' ')  # 替换 > 为空格
    ],
    # JSON 块提取模式
    "json_block_pattern": r'```\{([\s\S]*?)\}```'
}
### ***** Config *****

# 处理JSON内容，应用所有替换模式
def process_content(content, patterns):
    if not isinstance(content, str):
        return content
    # 应用所有预处理模式
    for pattern, replacement in patterns:
        content = re.sub(pattern, replacement, content)
    return content

# 提取 JSON 块
def extract_json_block(text):
    """
    Extracts JSON content between ```{ and }``` markers.
    Returns the content without the markers.
    """
    if not isinstance(text, str):
        return text
    
    pattern = config["json_block_pattern"]
    match = re.search(pattern, text, re.DOTALL)  # re.DOTALL allows . to match newlines
    if match:
        return match.group(1).strip()
    return text

# 计算 Token Level 的 TP、FP、FN
def calculate_metrics(row):
    def tokenize(text):
        if not isinstance(text, str) or not text.strip():
            return []
        tokens = []
        for item in re.split(r'[:,]', text.strip()):
            tokens.extend([t.strip() for t in item.split() if t.strip()])
        return tokens

    groundtruth_tokens = tokenize(row[config["output_columns"]["content1_column"]])
    extracted_tokens = tokenize(row[config["output_columns"]["content2_column"]])

    # If both are empty, return perfect metrics
    if not groundtruth_tokens and not extracted_tokens:
        return 1, 0, 0  # TP, FP, FN

    true_positives = 0
    used_positions = set()

    for i, gt_token in enumerate(groundtruth_tokens):
        for j, ex_token in enumerate(extracted_tokens):
            if j not in used_positions and gt_token == ex_token:
                true_positives += 1
                used_positions.add(j)
                break

    false_positives = len(extracted_tokens) - true_positives
    false_negatives = len(groundtruth_tokens) - true_positives

    return true_positives, false_positives, false_negatives

# 计算 Character Level 的 TP、FP、FN
def calculate_char_metrics(row):
    # Get strings
    groundtruth = str(row[config["output_columns"]["content1_column"]]).strip()
    extracted = str(row[config["output_columns"]["content2_column"]]).strip()
    
    # If both are empty, return perfect metrics
    if not groundtruth and not extracted:
        return 1, 0, 0  # TP, FP, FN
    
    # Create character count dictionaries
    gt_chars = {}
    ex_chars = {}
    
    # Count occurrences of each character (ignoring whitespace and punctuation)
    for char in groundtruth:
        if char.strip() and not char in ',./:;(){}[]':
            gt_chars[char] = gt_chars.get(char, 0) + 1
            
    for char in extracted:
        if char.strip() and not char in ',./:;(){}[]':
            ex_chars[char] = ex_chars.get(char, 0) + 1
    
    # Calculate true positives
    true_positives = 0
    for char, count in gt_chars.items():
        if char in ex_chars:
            true_positives += min(count, ex_chars[char])
    
    # Calculate false positives and false negatives
    false_positives = sum(ex_chars.values()) - true_positives
    false_negatives = sum(gt_chars.values()) - true_positives
    
    return true_positives, false_positives, false_negatives

# 计算整体指标
def calculate_overall_metrics(df):
    total_tp, total_fp, total_fn = 0, 0, 0
    total_char_tp, total_char_fp, total_char_fn = 0, 0, 0

    for _, row in df.iterrows():
        # Token-level metrics
        tp, fp, fn = calculate_metrics(row)
        total_tp += tp
        total_fp += fp
        total_fn += fn

        # Character-level metrics
        char_tp, char_fp, char_fn = calculate_char_metrics(row)
        total_char_tp += char_tp
        total_char_fp += char_fp
        total_char_fn += char_fn

    # Calculate overall token-level metrics
    token_precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    token_recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
    token_f1 = (2 * token_precision * token_recall) / (token_precision + token_recall) if (token_precision + token_recall) > 0 else 0
    token_accuracy = total_tp / (total_tp + total_fp + total_fn) if (total_tp + total_fp + total_fn) > 0 else 0

    # Calculate overall character-level metrics
    char_precision = total_char_tp / (total_char_tp + total_char_fp) if (total_char_tp + total_char_fp) > 0 else 0
    char_recall = total_char_tp / (total_char_tp + total_char_fn) if (total_char_tp + total_char_fn) > 0 else 0
    char_f1 = (2 * char_precision * char_recall) / (char_precision + char_recall) if (char_precision + char_recall) > 0 else 0
    char_accuracy = total_char_tp / (total_char_tp + total_char_fp + total_char_fn) if (total_char_tp + total_char_fp + total_char_fn) > 0 else 0

    return {
        "Token_Precision": token_precision,
        "Token_Recall": token_recall,
        "Token_F1": token_f1,
        "Token_Accuracy": token_accuracy,
        "Token_TP": total_tp,
        "Token_FP": total_fp,
        "Token_FN": total_fn,
        "Char_Precision": char_precision,
        "Char_Recall": char_recall,
        "Char_F1": char_f1,
        "Char_Accuracy": char_accuracy,
        "Char_TP": total_char_tp,
        "Char_FP": total_char_fp,
        "Char_FN": total_char_fn
    }

# 主函数
def main():
    # 从第一个目录读取数据
    data = []
    for filename in os.listdir(config["json_dir1"]):
        if filename.endswith(".json"):
            file_path = os.path.join(config["json_dir1"], filename)
            with open(file_path, "r", encoding="utf-8") as file:
                content = file.read()
                # 解析 JSON 内容，确保 Unicode 转义字符被正确解码
                try:
                    content = json.loads(content)  # 解析 JSON 内容
                    content = json.dumps(content, ensure_ascii=False)  # 将 JSON 转换为字符串，确保 Unicode 字符被正确显示
                except json.JSONDecodeError:
                    # 如果 JSON 解析失败，直接使用原始内容
                    pass
                # 应用通用替换模式
                content = process_content(content, config["preprocess_patterns"])  # 应用通用替换模式
                # 去掉文件名中的 .json 后缀
                filename_without_ext = os.path.splitext(filename)[0]
                # 创建只有两列的行
                row = [filename_without_ext, content, "", ""]
                data.append(row)

    # 创建包含四列的 DataFrame
    df = pd.DataFrame(data, columns=[
        config["output_columns"]["filename1_column"],
        config["output_columns"]["content1_column"],
        config["output_columns"]["filename2_column"],
        config["output_columns"]["content2_column"]
    ])

    # 从第二个目录读取数据填充第三和第四列
    row_index = 0
    for filename in os.listdir(config["json_dir2"]):
        if filename.endswith(".json") and row_index < len(df):
            file_path = os.path.join(config["json_dir2"], filename)
            with open(file_path, "r", encoding="utf-8") as file:
                content = file.read()
                # 解析 JSON 内容，确保 Unicode 转义字符被正确解码
                try:
                    content = json.loads(content)  # 解析 JSON 内容
                    content = json.dumps(content, ensure_ascii=False)  # 将 JSON 转换为字符串，确保 Unicode 字符被正确显示
                except json.JSONDecodeError:
                    # 如果 JSON 解析失败，直接使用原始内容
                    pass
                # 应用通用替换模式
                content = process_content(content, config["preprocess_patterns"])  # 应用通用替换模式
                # 去掉文件名中的 .json 后缀
                filename_without_ext = os.path.splitext(filename)[0]
                # 更新当前行的第三和第四列
                df.at[row_index, config["output_columns"]["filename2_column"]] = filename_without_ext
                df.at[row_index, config["output_columns"]["content2_column"]] = content
                row_index += 1

    # 对第二列应用额外的替换规则
    df[config["output_columns"]["content1_column"]] = df[config["output_columns"]["content1_column"]].apply(
        lambda x: process_content(x, config["preprocess_patterns2"])  # 先应用第二列专用规则
    )
    df[config["output_columns"]["content1_column"]] = df[config["output_columns"]["content1_column"]].apply(
        lambda x: process_content(x, config["preprocess_patterns"])  # 再应用通用替换模式
    )

    # 对第二列和第四列应用 preprocess_patterns3
    df[config["output_columns"]["content1_column"]] = df[config["output_columns"]["content1_column"]].apply(
        lambda x: process_content(x, config["preprocess_patterns3"])  # 应用 preprocess_patterns3
    )
    df[config["output_columns"]["content2_column"]] = df[config["output_columns"]["content2_column"]].apply(
        lambda x: process_content(x, config["preprocess_patterns3"])  # 应用 preprocess_patterns3
    )

    # 计算每一行的 Token Level 和 Character Level 的 TP、FP、FN
    df["Token_TP"] = 0
    df["Token_FP"] = 0
    df["Token_FN"] = 0
    df["Char_TP"] = 0
    df["Char_FP"] = 0
    df["Char_FN"] = 0

    for index, row in df.iterrows():
        # 计算 Token Level 的 TP、FP、FN
        token_tp, token_fp, token_fn = calculate_metrics(row)
        df.at[index, "Token_TP"] = token_tp
        df.at[index, "Token_FP"] = token_fp
        df.at[index, "Token_FN"] = token_fn
        
        # 计算 Character Level 的 TP、FP、FN
        char_tp, char_fp, char_fn = calculate_char_metrics(row)
        df.at[index, "Char_TP"] = char_tp
        df.at[index, "Char_FP"] = char_fp
        df.at[index, "Char_FN"] = char_fn

    # 计算整体指标
    overall_metrics = calculate_overall_metrics(df)

    # 输出整体指标
    print("\nOverall Results:")
    print("Token-level Metrics:")
    print(f"Token Precision: {overall_metrics['Token_Precision']:.4f}")
    print(f"Token Recall: {overall_metrics['Token_Recall']:.4f}")
    print(f"Token F1-Score: {overall_metrics['Token_F1']:.4f}")
    print(f"Token Accuracy: {overall_metrics['Token_Accuracy']:.4f}")
    print(f"Token TP: {overall_metrics['Token_TP']}, FP: {overall_metrics['Token_FP']}, FN: {overall_metrics['Token_FN']}")

    print("\nCharacter-level Metrics:")
    print(f"Char Precision: {overall_metrics['Char_Precision']:.4f}")
    print(f"Char Recall: {overall_metrics['Char_Recall']:.4f}")
    print(f"Char F1-Score: {overall_metrics['Char_F1']:.4f}")
    print(f"Char Accuracy: {overall_metrics['Char_Accuracy']:.4f}")
    print(f"Char TP: {overall_metrics['Char_TP']}, FP: {overall_metrics['Char_FP']}, FN: {overall_metrics['Char_FN']}")

    # 保存为 CSV，确保内容被正确引用
    df.to_csv(config["csv_output_path"], index=False, encoding="utf-8", quoting=csv.QUOTE_ALL)
    print(f"CSV 文件已保存至 {config['csv_output_path']}")

if __name__ == "__main__":
    main()

# Abstarct Pre-processing

In [74]:
import os
import json
import csv
import pandas as pd
import re

# Configurations
CONFIG = {
    "folder_path": "/Users/zixuanzhao/Desktop/Metrics/Structured-Info/Results/Structural-Abstract/GPT-3.5",
    "output_csv_path": "/Users/zixuanzhao/Desktop/Metrics/Structured-Info/Results/Structural-Abstract/GPT-3.5.csv",
    "possible_keys": [
        # "processing_route", "processing route", "processingroute", "Processing_route", "Processing route", "ProcessingRoute", "PROCESSING_ROUTE", "PROCESSING ROUTE", "PROCESSINGROUTE", "Processing_Route"
        # "main_idea", "main idea", "mainidea", "Main_idea", "Main idea", "MainIdea", "MAIN_IDEA", "MAIN IDEA", "MAINIDEA"
        # "research_object", "research object", "researchobject", "Research_object", "Research object", "Research_Object", "ResearchObject", "RESEARCH_OBJECT", "RESEARCH OBJECT", "RESEARCHOBJECT"
        "investigated_performance", "investigated performance", "investigatedperformance", "Investigated_performance", "Investigated performance", "Investigated_Performance", "InvestigatedPerformance", "INVESTIGATED_PERFORMANCE", "INVESTIGATED PERFORMANCE", "INVESTIGATEDPERFORMANCE"
         "mechanical_properties", "mechanical properties", "mechanicalproperties", "Mechanical_properties", "Mechanical properties", "Mechanical_Properties", "MechanicalProperties", "MECHANICAL_PROPERTIES", "MECHANICAL PROPERTIES", "MECHANICALPROPERTIES"
    ],
    
    "patterns": [
    # Patterns for values with quotes
    # r'"processing_route"\s*:\s*"([^"]+)"',
    # r'"processing route"\s*:\s*"([^"]+)"',
    # r'"Processing_route"\s*:\s*"([^"]+)"',
    # r'"Processing route"\s*:\s*"([^"]+)"',
    # r'"PROCESSING_ROUTE"\s*:\s*"([^"]+)"',
    # r'"PROCESSING ROUTE"\s*:\s*"([^"]+)"',
    # r'"processingroute"\s*:\s*"([^"]+)"',
    # r'"ProcessingRoute"\s*:\s*"([^"]+)"',
    # r'"PROCESSINGROUTE"\s*:\s*"([^"]+)"',
    # r'"Processing_Route"\s*:\s*"([^"]+)"',
    # r'"Processing_route"\s*:\s*"([^"]+)"',

    # r'"main idea"\s*:\s*"([^"]+)"',
    # r'"Main_idea"\s*:\s*"([^"]+)"',
    # r'"Main idea"\s*:\s*"([^"]+)"',
    # r'"MAIN_IDEA"\s*:\s*"([^"]+)"',
    # r'"main_idea"\s*:\s*"([^"]+)"',
    # r'"MAIN IDEA"\s*:\s*"([^"]+)"',
    # r'"mainidea"\s*:\s*"([^"]+)"',
    # r'"MainIdea"\s*:\s*"([^"]+)"',
    # r'"MAINIDEA"\s*:\s*"([^"]+)"',

    # r'"research_object"\s*:\s*"([^"]+)"',
    # r'"research object"\s*:\s*"([^"]+)"',
    # r'"Research_object"\s*:\s*"([^"]+)"',
    # r'"Research object"\s*:\s*"([^"]+)"',
    # r'"RESEARCH_OBJECT"\s*:\s*"([^"]+)"',
    # r'"RESEARCH OBJECT"\s*:\s*"([^"]+)"',
    # r'"researchobject"\s*:\s*"([^"]+)"',
    # r'"ResearchObject"\s*:\s*"([^"]+)"',
    # r'"RESEARCHOBJECT"\s*:\s*"([^"]+)"',
    # r'"Research_Object"\s*:\s*"([^"]+)"',

    r'"investigated_performance"\s*:\s*"([^"]+)"',
    r'"investigated performance"\s*:\s*"([^"]+)"',
    r'"Investigated_performance"\s*:\s*"([^"]+)"',
    r'"Investigated performance"\s*:\s*"([^"]+)"',
    r'"INVESTIGATED_PERFORMANCE"\s*:\s*"([^"]+)"',
    r'"INVESTIGATED PERFORMANCE"\s*:\s*"([^"]+)"',
    r'"investigatedperformance"\s*:\s*"([^"]+)"',
    r'"InvestigatedPerformance"\s*:\s*"([^"]+)"',
    r'"INVESTIGATEDPERFORMANCE"\s*:\s*"([^"]+)"',

    r'"mechanical_properties"\s*:\s*"([^"]+)"',
    r'"mechanical properties"\s*:\s*"([^"]+)"',
    r'"Mechanical_properties"\s*:\s*"([^"]+)"',
    r'"Mechanical properties"\s*:\s*"([^"]+)"',
    r'"MECHANICAL_PROPERTIES"\s*:\s*"([^"]+)"',
    r'"MECHANICAL PROPERTIES"\s*:\s*"([^"]+)"',
    r'"mechanicalproperties"\s*:\s*"([^"]+)"',
    r'"MechanicalProperties"\s*:\s*"([^"]+)"',
    r'"MECHANICALPROPERTIES"\s*:\s*"([^"]+)"',

    # Patterns for values without quotes
    # r'"processing_route"\s*:\s*([^"\n,]+)',
    # r'"processing route"\s*:\s*([^"\n,]+)',
    # r'"Processing_route"\s*:\s*([^"\n,]+)',
    # r'"Processing route"\s*:\s*([^"\n,]+)',
    # r'"PROCESSING_ROUTE"\s*:\s*([^"\n,]+)',
    # r'"PROCESSING ROUTE"\s*:\s*([^"\n,]+)',
    # r'"processingroute"\s*:\s*([^"\n,]+)',
    # r'"ProcessingRoute"\s*:\s*([^"\n,]+)',
    # r'"PROCESSINGROUTE"\s*:\s*([^"\n,]+)',
    # r'"Processing_Route"\s*:\s*([^"\n,]+)',
    # r'"Processing_route"\s*:\s*([^"\n,]+)',

    # r'"main idea"\s*:\s*([^"\n,]+)',
    # r'"Main_idea"\s*:\s*([^"\n,]+)',
    # r'"Main idea"\s*:\s*([^"\n,]+)',
    # r'"MAIN_IDEA"\s*:\s*([^"\n,]+)',
    # r'"main_idea"\s*:\s*([^"\n,]+)',
    # r'"MAIN IDEA"\s*:\s*([^"\n,]+)',
    # r'"mainidea"\s*:\s*([^"\n,]+)',
    # r'"MainIdea"\s*:\s*([^"\n,]+)',
    # r'"MAINIDEA"\s*:\s*([^"\n,]+)',

    # r'"research_object"\s*:\s*([^"\n,]+)',
    # r'"research object"\s*:\s*([^"\n,]+)',
    # r'"Research_object"\s*:\s*([^"\n,]+)',
    # r'"Research object"\s*:\s*([^"\n,]+)',
    # r'"RESEARCH_OBJECT"\s*:\s*([^"\n,]+)',
    # r'"RESEARCH OBJECT"\s*:\s*([^"\n,]+)',
    # r'"researchobject"\s*:\s*([^"\n,]+)',
    # r'"ResearchObject"\s*:\s*([^"\n,]+)',
    # r'"RESEARCHOBJECT"\s*:\s*([^"\n,]+)',
    # r'"Research_Object"\s*:\s*([^"\n,]+)',

    r'"investigated_performance"\s*:\s*([^"\n,]+)',
    r'"investigated performance"\s*:\s*([^"\n,]+)',
    r'"Investigated_performance"\s*:\s*([^"\n,]+)',
    r'"Investigated performance"\s*:\s*([^"\n,]+)',
    r'"INVESTIGATED_PERFORMANCE"\s*:\s*([^"\n,]+)',
    r'"INVESTIGATED PERFORMANCE"\s*:\s*([^"\n,]+)',
    r'"investigatedperformance"\s*:\s*([^"\n,]+)',
    r'"InvestigatedPerformance"\s*:\s*([^"\n,]+)',
    r'"INVESTIGATEDPERFORMANCE"\s*:\s*([^"\n,]+)',

    r'"mechanical_properties"\s*:\s*([^"\n,]+)',
    r'"mechanical properties"\s*:\s*([^"\n,]+)',
    r'"Mechanical_properties"\s*:\s*([^"\n,]+)',
    r'"Mechanical properties"\s*:\s*([^"\n,]+)',
    r'"MECHANICAL_PROPERTIES"\s*:\s*([^"\n,]+)',
    r'"MECHANICAL PROPERTIES"\s*:\s*([^"\n,]+)',
    r'"mechanicalproperties"\s*:\s*([^"\n,]+)',
    r'"MechanicalProperties"\s*:\s*([^"\n,]+)',
    r'"MECHANICALPROPERTIES"\s*:\s*([^"\n,]+)'
    ]
}

def extract_fields_from_json(folder_path, output_csv_path, possible_keys, patterns):
    """
    Extract fields from JSON files in a folder and save to CSV.
    
    Parameters:
    folder_path (str): Path to the folder containing JSON files
    output_csv_path (str): Path where the CSV file will be saved
    possible_keys (list): List of possible keys to extract
    patterns (list): List of regex patterns to extract values
    """
    data = []
    total_files = 0
    processed_files = 0
    error_files = []

    files = sorted(os.listdir(folder_path))
    
    for filename in files:
        if filename.endswith('.json'):
            total_files += 1
            file_path = os.path.join(folder_path, filename)
            try:
                extracted_info = extract_fields_from_file(file_path, possible_keys, patterns)
                doi = filename.replace('.json', '')
                data.append({"DOI": doi, "Extracted Info": extracted_info})
                processed_files += 1
            except Exception as e:
                error_files.append(filename)
                print(f"Error processing file {filename}: {str(e)}")
    
    df = pd.DataFrame(data)
    df.to_csv(output_csv_path, index=False, quoting=csv.QUOTE_ALL)
    
    print(f"Found {total_files} JSON files")
    print(f"Successfully processed {processed_files} files")
    print(f"Failed to process {len(error_files)} files")
    if error_files:
        print("Files with errors:")
        for err_file in error_files[:10]:
            print(f"  - {err_file}")
        if len(error_files) > 10:
            print(f"  ... and {len(error_files) - 10} more")
    
    return df

def extract_fields_from_file(file_path, possible_keys, patterns):
    """
    Extract fields from a JSON file using multiple methods.
    
    Parameters:
    file_path (str): Path to the JSON file
    possible_keys (list): List of possible keys to extract
    patterns (list): List of regex patterns to extract values
    
    Returns:
    str: Extracted information
    """
    extracted_info = ""
    
    # Method 1: Standard JSON parsing
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            json_data = json.loads(content)
            for key in possible_keys:
                if key in json_data:
                    value = json_data[key]
                    if isinstance(value, list):
                        extracted_info = ", ".join(value)  # Join list items into a single string
                    else:
                        extracted_info = value
                    break
    except:
        pass
    
    # Method 2: Clean content and retry JSON parsing
    if not extracted_info:
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                clean_content = re.sub(r'```(json)?|```', '', content).strip()
                json_data = json.loads(clean_content)
                for key in possible_keys:
                    if key in json_data:
                        value = json_data[key]
                        if isinstance(value, list):
                            extracted_info = ", ".join(value)
                        else:
                            extracted_info = value
                        break
        except:
            pass
    
    # Method 3: Regular expression to directly extract values
    if not extracted_info:
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                for pattern in patterns:
                    match = re.search(pattern, content)
                    if match:
                        extracted_info = match.group(1).strip()
                        break
        except:
            pass
    
    return extracted_info

if __name__ == "__main__":
    result_df = extract_fields_from_json(
        CONFIG["folder_path"],
        CONFIG["output_csv_path"],
        CONFIG["possible_keys"],
        CONFIG["patterns"]
    )
    if not result_df.empty:
        print("\nFirst few entries:")
        print(result_df.head())

Found 92 JSON files
Successfully processed 92 files
Failed to process 0 files

First few entries:
                          DOI  \
0   10.1007-s00170-018-1931-z   
1  10.1007-s00170-019-03789-w   
2  10.1007-s00170-020-05072-9   
3  10.1007-s00170-020-05242-9   
4  10.1007-s00170-023-11237-z   

                                      Extracted Info  
0  {'Tool_life_increase': {'Uncoated': '54%', 'Ti...  
1  Microstructure and micromechanical properties ...  
2  {'Initial_density': {'High': 'above 99%', 'Low...  
3  {'Ultimate_Tensile_Strength': '315 MPa', 'Micr...  
4                   Creep properties at 900°C/80 MPa  


# Abstarct Processing

In [329]:
import os
import pandas as pd
import re

### ***** Config *****
config = {
    # 文件路径配置
    "xlsx_path": "/Users/zixuanzhao/Desktop/Metrics/Structured-Info/Comparsion/Structural Abstract/Investigated_performance.xlsx",  # 输入的 Excel 文件路径
    "output_xlsx_path": "/Users/zixuanzhao/Desktop/Metrics/Structured-Info/Comparsion/Structural Abstract/Investigated_performance/SparkMax.xlsx",  # 输出的 Excel 文件路径
    # 列名配置
    "doi_column": "DOI",  # DOI 列名
    "groundtruth_column": "GroundTruth",  # GroundTruth 列名
    "model_column": "SparkMax",  # 模型列名（可以根据需要修改）
    # 需要替换的内容模式
    "preprocess_patterns": [
        (r'Here is the JSON output.*?(format|input)', ''),  # 去掉 "Here is the JSON output" 开头到 "format" 为止的部分
        (r'Here is the output.*?(format|input)', ''),
        (r'Here is.*?(format|input)', '')
    ],
    # 停用词列表（可以根据需要扩展）
    "stop_words": [
        "a", "an", "the", "and", "or", "but", "if", "then", "else", "when", "at", "by", "for", "in", "of", "on", "to", "with", "as", "about", 
        "after", "before", "during", "until", "above", "below", "from", "up", "down", "out", "over", "under", "again", "further", "once", "here", 
        "there", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", 
        "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now", "new", "newly", "newer", "newest", 
        "method", "methods", "methodology", "methodological", "improving", "improve", "improves", "improved", "improvement", "improvements", 
        "significant", "significance", "significantly", "proposed", "propose", "proposes", "proposing", "proposal", "proposals", "effect", 
        "effects", "effective", "effectively", "result", "results", "resulted", "resulting", "show", "shows", "showed", "showing", "find", 
        "finds", "found", "finding", "findings", "study", "studies", "studied", "studying", "analysis", "analyses", "analyze", "analyzes", 
        "analyzed", "analyzing", "research", "researches", "researched", "researching", "development", "developments", "develop", "develops", 
        "developed", "developing", "process", "processes", "processed", "processing", "approach", "approaches", "approached", "approaching", 
        "technique", "techniques", "strategy", "strategies", "system", "systems", "model", "models", "framework", "frameworks", "investigating", 
        "investigate", "investigates", "investigated", "investigation", "investigations", "revealing", "reveal", "reveals", "revealed", 
        "revelation", "influence", "influences", "influenced", "influencing", "exhibits", "exhibit", "exhibited", "exhibiting", "exhibition", 
        "this", "demonstrates", "demonstrate", "demonstrated", "demonstrating", "demonstration", "indicates", "indicate", "indicated", 
        "indicating", "indication", "suggests", "suggest", "suggested", "suggesting", "suggestion", "observes", "observe", "observed", 
        "observing", "observation", "highlights", "highlight", "highlighted", "highlighting", "confirms", "confirm", "confirmed", "confirming", 
        "confirmation", "proves", "prove", "proved", "proving", "proof", "explores", "explore", "explored", "exploring", "exploration", 
        "discusses", "discuss", "discussed", "discussing", "discussion", "examines", "examine", "examined", "examining", "examination", 
        "evaluates", "evaluate", "evaluated", "evaluating", "evaluation", "assesses", "assess", "assessed", "assessing", "assessment", "reports", 
        "report", "reported", "reporting", "presents", "present", "presented", "presenting", "presentation", "aimed", "aim", "aims", "aiming", 
        "used", "use", "uses", "using", "usage", "applying", "apply", "applies", "applied", "application", "focusing", "focus", "focuses", 
        "focused", "role", "roles", "introduces", "introduce", "introduced", "introducing", "introduction", "that", "designed", "design", 
        "designs", "designing", "good", "well", "better", "best", "achieving", "achieve", "achieves", "achieved", "achievement", "while", 
        "maintaining", "maintain", "maintains", "maintained", "maintenance", "through", "noting", "note", "notes", "noted", "due", "whereas", 
        "are", "is", "was", "were", "be", "being", "been", "affect", "affects", "affected", "affecting", "critical", "critically", "changes", 
        "change", "changed", "changing", "its", "adding", "add", "adds", "added", "addition", "via", "lead to", "leads to", "led to", "resulting in",
        "explain", "explains", "explained", "explaining", "explanation", "cause", "causes", "caused", "causing", "utilize", "utilizes", "utilized", "utilizing"
    ]
}
### ***** Config *****

def process_content(content, patterns):
    if not isinstance(content, str):
        return content
    # 应用所有预处理模式
    for pattern, replacement in patterns:
        content = re.sub(pattern, replacement, content)
    return content

def remove_stopwords_and_tokenize(text):
    if not isinstance(text, str) or not text.strip():
        return []
    
    # 去除标点符号
    text = re.sub(r'[^\w\s]', '', text)  # 移除非字母、数字和空格的字符
    
    # 将文本统一转换为小写
    text = text.lower()
    
    # 分词并过滤停用词
    tokens = []
    for item in re.split(r'[:,]', text.strip()):
        tokens.extend([t.strip() for t in item.split() if t.strip() and t.lower() not in config["stop_words"]])
    return tokens

def calculate_metrics(ground_truth, extraction):
    groundtruth_tokens = remove_stopwords_and_tokenize(ground_truth)
    extracted_tokens = remove_stopwords_and_tokenize(extraction)

    # If both are empty, return perfect metrics
    if not groundtruth_tokens and not extracted_tokens:
        return 1, 0, 0  # TP, FP, FN

    true_positives = 0
    used_positions = set()

    for i, gt_token in enumerate(groundtruth_tokens):
        for j, ex_token in enumerate(extracted_tokens):
            if j not in used_positions and gt_token == ex_token:
                true_positives += 1
                used_positions.add(j)
                break

    false_positives = len(extracted_tokens) - true_positives
    false_negatives = len(groundtruth_tokens) - true_positives

    return true_positives, false_positives, false_negatives

def calculate_char_metrics(ground_truth, extraction):
    # Get strings
    groundtruth = str(ground_truth).strip()
    extracted = str(extraction).strip()
    
    # If both are empty, return perfect metrics
    if not groundtruth and not extracted:
        return 1, 0, 0  # TP, FP, FN
    
    # Create character count dictionaries
    gt_chars = {}
    ex_chars = {}
    
    # Count occurrences of each character (ignoring whitespace and punctuation)
    for char in groundtruth:
        if char.strip() and not char in ',./:;(){}[]':
            gt_chars[char] = gt_chars.get(char, 0) + 1
            
    for char in extracted:
        if char.strip() and not char in ',./:;(){}[]':
            ex_chars[char] = ex_chars.get(char, 0) + 1
    
    # Calculate true positives
    true_positives = 0
    for char, count in gt_chars.items():
        if char in ex_chars:
            true_positives += min(count, ex_chars[char])
    
    # Calculate false positives and false negatives
    false_positives = sum(ex_chars.values()) - true_positives
    false_negatives = sum(gt_chars.values()) - true_positives
    
    return true_positives, false_positives, false_negatives

def main():
    # 读取 Excel 文件
    df = pd.read_excel(config["xlsx_path"])

    # 确保 DOI 列、GroundTruth 列和模型列存在
    required_columns = [config["doi_column"], config["groundtruth_column"], config["model_column"]]
    if not all(column in df.columns for column in required_columns):
        print(f"Error: Required columns {required_columns} not found in the Excel file.")
        return

    # 对 GroundTruth 列和模型列进行预处理
    df[config["groundtruth_column"]] = df[config["groundtruth_column"]].apply(
        lambda x: process_content(x, config["preprocess_patterns"])
    )
    df[config["model_column"]] = df[config["model_column"]].apply(
        lambda x: process_content(x, config["preprocess_patterns"])
    )

    # 计算每一行的 Token Level 和 Character Level 的 TP、FP、FN
    token_tp_list, token_fp_list, token_fn_list = [], [], []
    char_tp_list, char_fp_list, char_fn_list = [], [], []

    # 去除停用词并转换为列表
    groundtruth_tokens_list = []
    model_tokens_list = []

    # 新增：存储每行的精确度指标
    row_metrics = []

    print("\n每行的评估指标:")
    print("-" * 80)

    for index, row in df.iterrows():
        ground_truth = row[config["groundtruth_column"]]
        extraction = row[config["model_column"]]
        doi = row[config["doi_column"]]

        # 去除停用词并转换为列表
        groundtruth_tokens = remove_stopwords_and_tokenize(ground_truth)
        model_tokens = remove_stopwords_and_tokenize(extraction)
        groundtruth_tokens_list.append(groundtruth_tokens)
        model_tokens_list.append(model_tokens)

        # 计算 Token Level 的 TP、FP、FN
        token_tp, token_fp, token_fn = calculate_metrics(ground_truth, extraction)
        token_tp_list.append(token_tp)
        token_fp_list.append(token_fp)
        token_fn_list.append(token_fn)

        # 计算 Character Level 的 TP、FP、FN
        char_tp, char_fp, char_fn = calculate_char_metrics(ground_truth, extraction)
        char_tp_list.append(char_tp)
        char_fp_list.append(char_fp)
        char_fn_list.append(char_fn)

        # 新增：计算每行的精确度指标
        # 计算 Token-level 指标
        token_precision = token_tp / (token_tp + token_fp) if (token_tp + token_fp) > 0 else 0
        token_recall = token_tp / (token_tp + token_fn) if (token_tp + token_fn) > 0 else 0
        token_f1 = (2 * token_precision * token_recall) / (token_precision + token_recall) if (token_precision + token_recall) > 0 else 0
        token_accuracy = token_tp / (token_tp + token_fp + token_fn) if (token_tp + token_fp + token_fn) > 0 else 0

        # 计算 Character-level 指标
        char_precision = char_tp / (char_tp + char_fp) if (char_tp + char_fp) > 0 else 0
        char_recall = char_tp / (char_tp + char_fn) if (char_tp + char_fn) > 0 else 0
        char_f1 = (2 * char_precision * char_recall) / (char_precision + char_recall) if (char_precision + char_recall) > 0 else 0
        char_accuracy = char_tp / (char_tp + char_fp + char_fn) if (char_tp + char_fp + char_fn) > 0 else 0

        row_metrics.append({
            "DOI": doi,
            "Token_Precision": token_precision,
            "Token_Recall": token_recall,
            "Token_F1": token_f1,
            "Token_Accuracy": token_accuracy,
            "Char_Precision": char_precision,
            "Char_Recall": char_recall,
            "Char_F1": char_f1,
            "Char_Accuracy": char_accuracy
        })
        
        # 新增：打印每行的指标
        # print(f"DOI: {doi}")
        # print(f"GroundTruth: {ground_truth}")
        # print(f"Extraction: {extraction}")
        # print("Token-level 指标:")
        # print(f"  Precision: {token_precision:.4f}, Recall: {token_recall:.4f}, F1: {token_f1:.4f}, Accuracy: {token_accuracy:.4f}")
        # print(f"  TP: {token_tp}, FP: {token_fp}, FN: {token_fn}")
        # print("Character-level 指标:")
        # print(f"  Precision: {char_precision:.4f}, Recall: {char_recall:.4f}, F1: {char_f1:.4f}, Accuracy: {char_accuracy:.4f}")
        # print(f"  TP: {char_tp}, FP: {char_fp}, FN: {char_fn}")
        # print("-" * 80)

    # 创建一个新的 DataFrame，包含 DOI 列、处理后的 GroundTruth 列和模型列
    result_df = pd.DataFrame({
        config["doi_column"]: df[config["doi_column"]],
        config["groundtruth_column"]: groundtruth_tokens_list,
        config["model_column"]: model_tokens_list,
        "Token_TP": token_tp_list,
        "Token_FP": token_fp_list,
        "Token_FN": token_fn_list,
        "Char_TP": char_tp_list,
        "Char_FP": char_fp_list,
        "Char_FN": char_fn_list
    })

    # 计算整体指标
    total_token_tp = sum(token_tp_list)
    total_token_fp = sum(token_fp_list)
    total_token_fn = sum(token_fn_list)
    total_char_tp = sum(char_tp_list)
    total_char_fp = sum(char_fp_list)
    total_char_fn = sum(char_fn_list)

    # 新增：计算整体精确度指标
    # 计算 Token-level 整体指标
    total_token_precision = total_token_tp / (total_token_tp + total_token_fp) if (total_token_tp + total_token_fp) > 0 else 0
    total_token_recall = total_token_tp / (total_token_tp + total_token_fn) if (total_token_tp + total_token_fn) > 0 else 0
    total_token_f1 = (2 * total_token_precision * total_token_recall) / (total_token_precision + total_token_recall) if (total_token_precision + total_token_recall) > 0 else 0
    total_token_accuracy = total_token_tp / (total_token_tp + total_token_fp + total_token_fn) if (total_token_tp + total_token_fp + total_token_fn) > 0 else 0

    # 计算 Character-level 整体指标
    total_char_precision = total_char_tp / (total_char_tp + total_char_fp) if (total_char_tp + total_char_fp) > 0 else 0
    total_char_recall = total_char_tp / (total_char_tp + total_char_fn) if (total_char_tp + total_char_fn) > 0 else 0
    total_char_f1 = (2 * total_char_precision * total_char_recall) / (total_char_precision + total_char_recall) if (total_char_precision + total_char_recall) > 0 else 0
    total_char_accuracy = total_char_tp / (total_char_tp + total_char_fp + total_char_fn) if (total_char_tp + total_char_fp + total_char_fn) > 0 else 0

    # 将整体指标添加到 DataFrame 的最后几行
    overall_metrics = pd.DataFrame({
        config["doi_column"]: ["Overall Metrics"],
        config["groundtruth_column"]: [""],
        config["model_column"]: [""],
        "Token_TP": [total_token_tp],
        "Token_FP": [total_token_fp],
        "Token_FN": [total_token_fn],
        "Char_TP": [total_char_tp],
        "Char_FP": [total_char_fp],
        "Char_FN": [total_char_fn]
    })
    result_df = pd.concat([result_df, overall_metrics], ignore_index=True)

    # 保存为 Excel 文件
    result_df.to_excel(config["output_xlsx_path"], index=False)
    print(f"Excel 文件已保存至 {config['output_xlsx_path']}")
    
    # 新增：打印整体的评估指标
    print("\nOverall Metrics:")
    print("Token-level Metrics:")
    print(f"Token Precision: {total_token_precision:.4f}")
    print(f"Token Recall: {total_token_recall:.4f}")
    print(f"Token F1-Score: {total_token_f1:.4f}")
    print(f"Token Accuracy: {total_token_accuracy:.4f}")
    print(f"Token TP: {total_token_tp}, FP: {total_token_fp}, FN: {total_token_fn}")

    print("\nCharacter-level Metrics:")
    print(f"Char Precision: {total_char_precision:.4f}")
    print(f"Char Recall: {total_char_recall:.4f}")
    print(f"Char F1-Score: {total_char_f1:.4f}")
    print(f"Char Accuracy: {total_char_accuracy:.4f}")
    print(f"Char TP: {total_char_tp}, FP: {total_char_fp}, FN: {total_char_fn}")

if __name__ == "__main__":
    main()


每行的评估指标:
--------------------------------------------------------------------------------
Excel 文件已保存至 /Users/zixuanzhao/Desktop/Metrics/Structured-Info/Comparsion/Structural Abstract/Investigated_performance/SparkMax.xlsx

Overall Metrics:
Token-level Metrics:
Token Precision: 0.5583
Token Recall: 0.8422
Token F1-Score: 0.6715
Token Accuracy: 0.5054
Token TP: 747, FP: 591, FN: 140

Character-level Metrics:
Char Precision: 0.6183
Char Recall: 0.9409
Char F1-Score: 0.7462
Char Accuracy: 0.5952
Char TP: 7037, FP: 4344, FN: 442


### Tokenizer

In [39]:
import pandas as pd
import tiktoken
from transformers import AutoTokenizer

# 读取表格
df = pd.read_csv("/Users/zixuanzhao/Desktop/数据治理/Metrics/Structured-Info/Results/Structural-Text/llama-3.3-70b-instruct.csv")  # 或 pd.read_excel("your_file.xlsx")

# 初始化 tokenizer（以 GPT-4 为例）
encoding = tiktoken.encoding_for_model("gpt-4o")

# 选择列名（比如 "Prompt"）
column_name = "Composition"
# column_name = "Heat_Treatment"
# column_name = "Deformation/Manufacturing_Technique"
# column_name = "Microstructure1"
# column_name = "Microstructure2"
# column_name = "Properties"
# column_name = "Performance"
# column_name = "Strengthening_Mechanism"

# ✅ 初始化总 token 计数器
total_tokens = 0

# 从第二行开始逐行计算
for i, text in enumerate(df[column_name].iloc[1:], start=2):
    if pd.notna(text):
        num_tokens = len(encoding.encode(text))
        total_tokens += num_tokens
        print(f"Row {i}: Token count = {num_tokens}")
        
# ✅ 打印总 token 数
print(f"\nTotal token count: {total_tokens}")

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

Row 2: Token count = 84
Row 3: Token count = 84
Row 4: Token count = 45
Row 5: Token count = 48
Row 6: Token count = 68
Row 7: Token count = 49
Row 8: Token count = 75
Row 9: Token count = 75
Row 10: Token count = 74
Row 11: Token count = 75
Row 12: Token count = 75
Row 13: Token count = 172
Row 14: Token count = 173
Row 15: Token count = 173
Row 16: Token count = 173
Row 17: Token count = 162
Row 18: Token count = 152
Row 19: Token count = 153
Row 20: Token count = 152
Row 21: Token count = 143
Row 22: Token count = 145
Row 23: Token count = 146
Row 24: Token count = 158
Row 25: Token count = 158
Row 26: Token count = 154
Row 27: Token count = 210
Row 28: Token count = 47
Row 29: Token count = 47
Row 30: Token count = 47
Row 31: Token count = 53
Row 32: Token count = 53
Row 33: Token count = 53
Row 34: Token count = 53
Row 35: Token count = 53
Row 36: Token count = 53
Row 37: Token count = 53
Row 38: Token count = 53
Row 39: Token count = 86
Row 40: Token count = 96
Row 41: Token coun

In [44]:
import os
import json

path1 = '/Users/zixuanzhao/Desktop/数据治理/Metrics/Unstructured-Info/Table/untitled folder'  # 输入目录
path2 = '/Users/zixuanzhao/Desktop/数据治理/Metrics/Unstructured-Info/Abstract2'        # 输出目录
output_filename = "merged_Table.json"

def read_file_with_encoding_fallback(file_path):
    # 尝试 utf-8 和 latin-1 两种编码
    for enc in ['utf-8', 'latin-1']:
        try:
            with open(file_path, "r", encoding=enc) as f:
                return f.read()
        except Exception:
            continue
    print(f"❌ 无法读取文件: {file_path}")
    return None

def main():
    all_data = []

    for file in os.listdir(path1):
        if file.endswith(".json") and not file.startswith("."):
            file_path = os.path.join(path1, file)
            content = read_file_with_encoding_fallback(file_path)
            if content is not None:
                all_data.append({"File": content})

    # 创建输出目录
    os.makedirs(path2, exist_ok=True)
    output_path = os.path.join(path2, output_filename)

    # 写入合并结果
    with open(output_path, "w", encoding="utf-8") as f_out:
        json.dump(all_data, f_out, indent=2, ensure_ascii=False)

    print(f"✅ 合并完成，共合并 {len(all_data)} 个文件，输出路径：{output_path}")

if __name__ == "__main__":
    main()

✅ 合并完成，共合并 243 个文件，输出路径：/Users/zixuanzhao/Desktop/数据治理/Metrics/Unstructured-Info/Abstract2/merged_Table.json


In [60]:
# import json
# import tiktoken

# # 设定 JSON 文件路径
file_path = "/Users/zixuanzhao/Desktop/数据治理/Metrics/Unstructured-Info/Combined/merged_OCR+LLM.json"

# # 初始化 tokenizer（以 GPT-4o 为例）
# encoding = tiktoken.encoding_for_model("gpt-4o")
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
# encoding = tiktoken.encoding_for_model("gpt-4-1106-preview")
# 读取 JSON 文件内容为纯文本
with open(file_path, "r", encoding="utf-8") as f:
    content = f.read()  # 不做 json.load，而是直接读取整个文本内容

# 计算 token 数
token_count = len(encoding.encode(content))

# 输出 token 数
print(f"✅ 文件 {file_path} 的总 token 数为: {token_count}")

✅ 文件 /Users/zixuanzhao/Desktop/数据治理/Metrics/Unstructured-Info/Combined/merged_OCR+LLM.json 的总 token 数为: 1517


In [59]:
import docx
import tiktoken

# Step 1: 加载 Word 文档
doc_path = "..."  # 修改为你实际文件路径
doc = docx.Document(doc_path)

# Step 2: 提取并合并所有段落文本
full_text = "\n".join([para.text for para in doc.paragraphs if para.text.strip()])

# Step 3: 初始化 tokenizer（GPT-4o）
encoding = tiktoken.encoding_for_model("gpt-4-1106-preview")

# Step 4: 计算 token 数
token_count = len(encoding.encode(full_text))

# 输出
print(f"✅ 总 token 数: {token_count}")

PackageNotFoundError: Package not found at '/Users/zixuanzhao/Desktop/数据治理/Metrics/Unstructured-Info/Combined/merged_OCR+LLM.json'